# SVMやRandom Forest の代わりに深層学習を利用する

ここでは、SVMやRandom Forestの代わりに、PyTorch を利用した深層学習を実施して、SVM等同様に、細胞状態を予測します。

## 深層学習の実行に必要な要素

深層学習は、SVM、RandomForest同様に機械学習の一種なので、雰囲気としては、 ``clf=DeepLearning()`` の様な形で書くことが可能です。ところが、今までのSVM等に比べて

* データの読み込みが、メモリに乗らない超大規模な場合を想定しているため、少しずつデータを読み込んで学習する（バッチ）部分を記載する必要がある。
* 学習方法として、訓練データ、テストデータだけでなく、訓練データを（真の）訓練データ（勾配を計算し、次に進む探索方法の利用）とバリデーションデータ（訓練データで出てきたモデルの精度を検証する）に分割する。
* （ハイパー）パラメータの数が多い。SVMでは、Cやgamma など数個だったが、深層学習では、ネットワークの組み方、学習方法、学習率など、多種多様なパラメータが存在しており、これらを指定する必要がある
* GPUを利用する場合は、GPUを利用するための記述が必要になる。

といった変更すべき点があるため、もう少しプログラムが複雑になります。とはいえ、本質的な流れは変わりません。

## ライブラリの読み込み

少々長くなりますが、利用するライブラリをまとめて読み込みます。

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
import torch.utils.data as data
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import os
import copy
import argparse
import time
import numpy as np
import pandas as pd

## データの読み込み

データの読み込みに関する関数が二段階に別れます。

第一に、学習開始前のデータ読み込みです。一旦すべてのデータを読み込んで、SVMの時同様に利用する特徴量の抽出、正規化といった前処理を実施します。
第二に、学習中のデータ読み込みです。こちらでは、深層学習ではすべてのデータを一括で利用するのではなく、「バッチ」と言われる単位ごとに読み込むことで徐々に学習を進めます。バッチを繰り返してすべてのデータを一度利用し終わると、「エポック」が終了します。この「エポック」を複数回繰り返すことで、徐々に学習を進めていきます。

以下のプログラムでは、make_dataset 関数が、学習前のデータの一覧表作成を担当しています。
学習中は、DatasetFoler (を呼び出しているYeastFeatureDataset)を利用しており、DatasetFolderの```__getitem__```バッチごとのデータ生成を行っています。この関数は、Pythonの配列アクセス時に呼び出される ```__getitem__``` 関数の上書きで、DatasetFolderクラスの配列にアクセスしようとした時に、自動的に呼び出されます。このため、プログラム中では、陽にgetitem関数を見ることはありませんので、ご注意ください。

### 学習前のデータの読み込み

まず、学習開始前のデータの読み込みです。

```read_csv```でファイルからデータを読み込み、特徴量の選択と正規化するところまでは、今まで通りです。SVM等と異なるのは、クラス（細胞の芽の大きさのグループ）の指定方法です。今までは、"no","small"... に 0,1,... を割り当てていましたが、今回は"no"を0次元目、"small"を1次元目、という形で各次元に割り当て、属するクラスに1を、それ以外は0として指定します。これは、深層学習には各サンプルの特徴量から、"no"である確率、"small"である確率などの4つの確率を出力させて、最終的に一番確率の高い値のクラスを予測値としよう、という「マルチラベル」を用いた作戦を取るためです。

In [2]:
def make_dataset(dir):
    features = []
    labels = []
    # データの読み込みと特徴量の選択
    dataset = pd.read_csv(os.path.join(dir, "yeast_his3.csv"))
    columns = ["C101", "C103", "C104", "C115", "A101", "A120", "A121", "A122", "A123"]
    cell_features_pre = dataset[["Cgroup"] + columns]
    cell_features = cell_features_pre[np.sum(cell_features_pre.isnull(), axis=1) == 0]
    X = cell_features[columns]
    groups = np.array(cell_features["Cgroup"])
    # 正規化
    X_norm = preprocessing.StandardScaler().fit_transform(X)
    # クラスの指定
    for i in range(len(groups)):
        group = groups[i]
        feature = X_norm[i]
        y = [0, 0, 0, 0]
        if group == "no":
            y = [1, 0, 0, 0]
        elif group == "small":
            y = [0, 1, 0, 0]
        elif group == "medium":
            y = [0, 0, 1, 0]
        elif group == "large":
            y = [0, 0, 0, 1]
        elif group == "complex":
            y = [0, 0, 0, 0]
        features.append(np.array(feature.astype(np.float32)))
        labels.append(np.array(y))
    return features, labels

### データの分割（Training, Validation, Test）

SVMでは、データを訓練(Training)データとテスト(Test)データの２つに分割し、訓練データを利用して学習、テストデータを利用して学習結果の予測精度を評価しました。深層学習も基本的に同様ですが、深層学習では、訓練データを更に（真の）訓練データとバリデーション（Validation）データに分割します。深層学習では、真の訓練データで学習を行いつつ、バリデーションデータで精度の確認を実施し、最終的にバリデーションデータで最も性能の良かったモデルを、最終的なモデルとして出力する構造をしているためです。
テストデータには学習に利用したサンプルが入っていてはいけないので、訓練データ内の分割を実施することになります。

以下の例では、Training:Validation:Test = 3:1:1 になるように分割をしています。

In [3]:
# データの読み込み
X, y = make_dataset("data")
# テストデータの分割
X_tmp, X_test, y_tmp, y_test = train_test_split(X, y, test_size = 0.20)
# 訓練データとValidationデータの分割
X_train, X_val, y_train, y_val = train_test_split(X_tmp, y_tmp, test_size = 0.25)

### 学習中のデータの読み込み（バッチごとの読み込み）

データは、以下のDatasetFolderクラスで格納されます。Xが特徴量、yがクラス（予測したい芽の大きさのグループ）です。
feature_datasets['train'] に訓練データ、feature_datasets['val'] にバリデーションデータ、feature_datasets['test']にテストデータが入ります。

In [4]:
class DatasetFolder(data.Dataset):
    def __init__(self, X, y):
        self.samples = X
        self.targets = y

    def __getitem__(self, index):
        sample = self.samples[index]
        target = self.targets[index]
        sample = torch.from_numpy(sample)
        target = torch.from_numpy(target)
        return sample, target

    def __len__(self):
        return len(self.samples)

feature_datasets = {
    'train':DatasetFolder(X_train, y_train),
    'val':DatasetFolder(X_val, y_val),
    'test': DatasetFolder(X_test, y_test)
}

確認してみましょう。訓練データの一つのサンプルに関して、9次元の特徴量ベクトル（正規化後のもの）とクラス情報が現れます。

In [5]:
sample, target = feature_datasets['train'][0]
print(sample)
print(target)

tensor([ 0.5275, -0.6105, -0.2206, -0.5376,  0.2268,  0.7377,  1.0948,  1.9232,
        -0.0617])
tensor([0, 0, 0, 1])


このDatasetFolderの情報は、学習時には1個1個ではなく、以下のdataloadersを通じて、バッチサイズ毎(以下の例では64サンプル)にまとめて読み込まれます。

In [6]:
# バッチサイズ分のデータを読み込む。
# 訓練データ（train）はshuffle=Trueを指定することで、
# データの順番をシャッフルし、読み込む順番をランダムにする。
# 他はシャッフルの必要なし。
batch_size=64
workers=0
dataloaders = {
    # 訓練データ
    'train': torch.utils.data.DataLoader(
        feature_datasets['train'],
        batch_size=batch_size,
        shuffle=True,
        num_workers=workers),
    # バリデーションデータ
    'val': torch.utils.data.DataLoader(
        feature_datasets['val'],
        batch_size=batch_size,
        shuffle=False,
        num_workers=workers),
    # テストデータ
    'test': torch.utils.data.DataLoader(
        feature_datasets['test'],
        batch_size=batch_size,
        shuffle=False,
        num_workers=workers)
}
dataset_sizes = {x: len(feature_datasets[x]) for x in ['train', 'val', 'test']}

## （深層）学習モデルの定義

深層学習のモデルの定義を（やっと）行います。ここでは、全く深い層ではなく、中間層1層だけのニューラルネットワークを組んで、学習してみます。

入力の特徴量数が9個、中間層のノードが9個、最終層がクラスを判別する4個(no, small, medium, large)のネットワークを考えます。設定は至ってシンプルで、層の名前と形式を定義する部分(以下の``__init__``内のLinear2つ)と、それらの組み合わせを与える部分(``forward``関数内)から構成されます。これで
特徴量9個→完全連結の層(Dense層と言われます)→出力層
という中間層１層を含む３層構造のネットワークが出来ます。活性化関数（各層の入力と出力の関係）として、ここではReLuを利用しています。

In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 9)
        self.fc2 = nn.Linear(9, 4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# ここから先は、作成したネットワークを、指定のデバイスに送るための内容。
# CPUではなく、GPUを利用したい場合は、"cuda" もしくは、"cuda:0" などと
# 設定を記載。
device_name = "cpu"
device = torch.device(device_name)
model = Net()
model = model.to(device)

### 学習ステップの定義

今までの説明で、基本的な深層学習の定義は終了していますが、最後に学習ステップ自身の定義を行います。プログラムは少し長いですが、エポックをforループでまわし、エポック終了と共に途中経過を表示するルーチンを書いてあるだけなので、定形の処理です。より高度な深層学習では、重みの更新を一部止めたり、ネットワーク自身の変更をしたりすることが可能ですが、ここでは最もシンプルな処理のみを記載します。

In [8]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    # 途中経過でモデル保存するための初期化
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    # 時間計測用
    end = time.time()

    print(model)
    print()

    for epoch in range(num_epochs):
        print('Epoch:{}/{}'.format(epoch, num_epochs - 1), end="")

        # 各エポックで訓練+バリデーションを実行
        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train(True)  # training mode
            else:
                model.train(False)  # evaluate mode

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                labels = labels.float()
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # 訓練のときだけ履歴を保持する
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, classnums = torch.max(labels, 1)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, classnums)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # 統計情報
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == classnums)

            # サンプル数で割って平均を求める
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('\t{} Loss: {:.4f} Acc: {:.4f} Time: {:.4f}'.format(phase, epoch_loss, epoch_acc, time.time()-end), end="")

            # 精度が改善したらモデルを保存する
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            end = time.time()

        print()

    time_elapsed = time.time() - since
    print()
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val acc: {:.4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

最後に、テストデータでの結果の評価を表示するための関数を定義します。

In [9]:
def print_test_accuracy(model, criterion, optimizer, phase):
    running_loss = 0.0
    running_corrects = 0
    model.train(False)

    for inputs, labels in dataloaders[phase]:
        labels = labels.float()
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # 訓練のときだけ履歴を保持する
        with torch.set_grad_enabled(phase == 'train'):
            outputs = model(inputs)
            _, classnums = torch.max(labels, 1)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, classnums)

        # 統計情報
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == classnums)

    # サンプル数で割って平均を求める
    epoch_loss = running_loss / dataset_sizes[phase]
    epoch_acc = running_corrects.double() / dataset_sizes[phase]
    print('On Test:\tLoss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

これで準備は終了です。

## 深層学習の実行

下記のコードで、これまでに定義したモデルを学習します。

In [10]:
epochs = 20
batch_size = 64
lr = 0.1
momentum = 0.9
outdir = "."

# 損失関数（クロスエントロピー）、
# パラメータの最適化方法、学習率の更新方法を定義。
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

# 実際の学習を実施
model = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=epochs)
# 学習が終わったら、結果を保存する。
torch.save(model.state_dict(), 'model.pkl')
# テストデータでの精度を求める
print_test_accuracy(model, criterion, optimizer, 'test')

Net(
  (fc1): Linear(in_features=9, out_features=9, bias=True)
  (fc2): Linear(in_features=9, out_features=4, bias=True)
)

Epoch:0/19	train Loss: 1.4481 Acc: 0.1802 Time: 0.0104	val Loss: 1.2733 Acc: 0.4533 Time: 0.0019
Epoch:1/19	train Loss: 1.2313 Acc: 0.4865 Time: 0.0080	val Loss: 1.1527 Acc: 0.5200 Time: 0.0019
Epoch:2/19	train Loss: 1.1167 Acc: 0.5180 Time: 0.0067	val Loss: 0.9860 Acc: 0.5600 Time: 0.0019
Epoch:3/19	train Loss: 0.9447 Acc: 0.5721 Time: 0.0047	val Loss: 0.8350 Acc: 0.6000 Time: 0.0014
Epoch:4/19	train Loss: 0.8672 Acc: 0.5405 Time: 0.0064	val Loss: 0.7750 Acc: 0.6000 Time: 0.0018
Epoch:5/19	train Loss: 0.7477 Acc: 0.6396 Time: 0.0058	val Loss: 0.7364 Acc: 0.6000 Time: 0.0014
Epoch:6/19	train Loss: 0.7092 Acc: 0.6532 Time: 0.0058	val Loss: 0.6221 Acc: 0.6667 Time: 0.0016
Epoch:7/19	train Loss: 0.6267 Acc: 0.7387 Time: 0.0076	val Loss: 0.5754 Acc: 0.7600 Time: 0.0018
Epoch:8/19	train Loss: 0.5522 Acc: 0.7748 Time: 0.0055	val Loss: 0.5622 Acc: 0.6800 Time: 0.0045
Epo

エポックが一つ終わるごとに、下記の情報を出力しています。
* 予測したクラスと正解のクラスの誤差（ロス）
    * 評価関数で定義しているクロスエントロピーの値
    * 訓練データにおけるロス train Loss と、評価データにおけるロスval Loss
* 予測精度（Accuracy; Acc)
* 実行時間

はじめの数エポックでは、訓練データでもバリデーションデータでも精度が6割くらいと高くありませんが、20エポック終わる頃には、9割近い精度を挙げています。

この精度が独立データ（テストデータ）でも同様かどうかが、最後に計算されており、精度が8割程度（実行の度に異なりますので、これらの値は、皆さんの手元では異なるかもしれません）であることがわかります。

今回の実行時の先頭に、学習時の主要なパラメータが記載されています。

```python
epochs = 20
batch_size = 64
lr = 0.1
momentum = 0.9
```

epochsはエポック数(エポックを何回繰り返すか)、batch_sizeは各バッチで利用するサンプル数、lr は学習率と呼ばれ、一回の学習でどれだけ一気に学習を進めるか、momentum は、今回利用している学習方法であるSGDにおいて利用するパラメータで、慣性を表している値です。いずれの値にも正解はなく、利用しているネットワークモデルや、データの種類に応じて、適切なパラメータを見つける必要があります。


以上で、SVMやRandom Forestの代わりに深層学習を利用する学習方法は終了です。次に、特徴量を抽出せずに、直接画像から学習する方法に進みましょう。